<a href="https://colab.research.google.com/github/CBravoR/AdvancedAnalyticsLabs/blob/master/notebooks/python/Lab_4_Preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Preprocessing

In this lab, we will set up a **data pipeline** in order to leave the data ready for analysis. Data preprocessing tends to be, by far, the most time-consuming step of the data science process. Errors in this step propagate to the model, so it is really important we do this correctly.

The goal of this step is to leave the date ready to apply models to it. 

Every problem has its own set of data preprocessing functions to apply it to, but we will focus on the ones most common in classification models. In general we want to:

1. Eliminate redundant variables.
2. Treat null values.
3. Treat outliers.
4. Remove correlated features.

For this goal we will use the excellent [```scikit-learn```](https://scikit-learn.org/stable/) package, which comes with most data-intensive operations. We will also use today (and during the rest of the module) the [```pandas```](https://pandas.pydata.org/) package, which allows for data handling in general.

## Reading the data

Let's download the Bankloan data. I have slightly modified it removing the PD and LGD information (as we will now estimate our own). As a reminder, it has the following variables:

- Customer: ID, or unique label, of the borrower (NOT predictive).
- Age: Age of the borrower in years.
- Education: Maximum education level the borrower reached.
1: Complete primary. 2: Completed Secondary. 3: Incomplete Higher Ed. 4: Complete Higher Ed. 5: With postgraduate studies (complete MSc or PhD).
- Employ: Years at current job.
- Address: Years at current address.
- Income: Income in ‘000s USD.
- Leverage: Debt/Income Ratio.
- CredDebt: Credit card standing debt.
- OthDebt: Other debt in ‘000s USD.
- MonthlyLoad: Monthly percentage from salary used to repay debts.
- Default: 1 If default has occurred, 0 if not (Target variable).

We will download it now directly from a link, using the more ubiquitous [```wget```](http://www.gnu.org/software/wget/) command. The command is

```
wget [-options] path
```

We need to add the options ```--no-check-certificate``` and ```--output-document=FILENAME``` so it downloads ok.

In [ ]:
!wget --no-check-certificate --output-document=Bankloan.csv 'https://docs.google.com/spreadsheets/d/1nUJ1fA5f1VeMvulknpsvxpy0GW3CekNnhgeLRK0WlDI/export?gid=1016776666&format=csv'

To check what we downloaded we can use the ```head``` OS command.

In [ ]:
!head Bankloan.csv

Now we will use Pandas to read the CSV file. The  function to do so is [```read_csv```](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html). We will store the results in a variable named ```bankloan_data```.

In [ ]:
import pandas as pd

bankloan_data = pd.read_csv('Bankloan.csv')

As a reminder, here are the summary statistics of the variables.

In [ ]:
bankloan_data.describe()

We can see there are a few null values in different variables and apparently an invalid outlier in Leverage.  Let's visualize the dataset using seaborn to get an idea of the distribution.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
%matplotlib inline

In [ ]:
sns.set(color_codes=True)

for col_id in bankloan_data.columns[np.r_[1,3:9]]:
    sns.displot(data = bankloan_data, x = col_id, hue = "Default", kind = 'kde')

Seaborn can do many sophisticated (and aestetically pleasing) 
graphs. Go to [the Gallery](https://seaborn.pydata.org/examples/index.html) for details and example code. 

There are severe outliers in OthDebt, Leverage, Creddebt and Income, but we don't know which ones are valid and which invalid. We will treat these now.

## Data Cleaning

Now we can focus on cleaning the data. Let's start with the easy part: removing null values. **Remember to check when an outlier is a missing value** (invalid outliers).

### Null values

The core function here will be Panda's [```fillna```](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.fillna.html). This allows to replace all null values (represented by None or NaN in Python) by a certain value. This also allows to set what the replacement will be with the ```value``` argument. See the help for details.

Remember the strategies to deal :

1. Keep: If the null values are a category by themselves. In this case, replace by something meaningful.

2. Delete: If the null values are too many **either by row or by column** then it is better to just drop the case or the variable.

3. Replace: If there are only a few missings for the variable or the row (<1% total), replace by the  replace the null values by the **median** for continous variables, and the **mode** for categorical values.

Let's study our dataset's null values. The [```isnull()```](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.isnull.html) function returns which elements in the dataframe are null. The [```any()```](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.any.html) function returns a list with whatever columns (default) or rows (passing ```axis = 1``` to the function) have any element with a boolean of true.

In [ ]:
bankloan_data.isnull().any()

We can see which columns have null values. Let's study them in further detail.

In [ ]:
null_columns = bankloan_data.columns[bankloan_data.isnull().any()]
bankloan_data[null_columns].isnull().sum()

Given the small number of cases for all variables except OthDebt, we can simply replace those values by the median.

Let's study OthDebt cases more in detail.

In [ ]:
bankloan_data.loc[bankloan_data.isnull().any(axis = 1), :]

We can see all cases in which OthDebt is null. We can also see the egregious outlier that Leverage has, with a value of 9999999. These values give us some hints about what's happening with OthDebt. Let's study the minimum and maximum of the variables.

In [ ]:
bankloan_data.OthDebt.describe()

Aha! Note anything? There are no values equal to 0! We can make a good guess that the null values are equal to zero. In real life we would call the sysadmins to confirm this, but in this case we can make a guess.

Then, we now should fix these values as follow:

1. Replace the leverage equal to 999999 by a null value.
2. Replace all OthDebt nulls by 0.
3. Replace the remaining null values by the median.

In [ ]:
# Replace invalid outlier.
bankloan_data.Leverage.values[bankloan_data.Leverage.values == 999999] = np.NaN

In [ ]:
# Fills out the null values with zeros. Inplace argument changes dataframe.
bankloan_data.OthDebt.fillna(value = 0, inplace=True)

In [ ]:
# Fill out remaining elements.
bankloan_data.fillna(bankloan_data.median(), inplace=True)

In [ ]:
bankloan_data.describe()

There are no more null values! We are now ready to study the distributions.

### Outlier cleaning

Cleaning outlier, on the other hand, requires a much more manual intervention. We know there are three variables we need to intervene in: Income, Creddebt, and OthDebt. In general, we would like to modify all cases where there is either a discontinuous distribution or a case that is outside of 3 to 6 standard deviations.

It is very important to use your judgement in this! Don't just cut in 3 stds, as that is too restrictive in an exponential distribution. If you want to get an idea of the values, then seaborn can help. The function ```displot``` with either the option ```hist``` or ```kde``` allows to check the distribution of values.

In [ ]:
sns.displot(bankloan_data['Income'], kind = 'hist')

In [ ]:
sns.displot(bankloan_data['Creddebt'], kind = 'kde')

In [ ]:
sns.displot(bankloan_data['OthDebt'], kind = 'kde')

All variables need to be trimmed from the right. Potential cutoffs are Income of 300, Creddebt of 15, and Othdebt of 30.

To actually cut the values, then we can simply write a proper expression in Pandas:

In [ ]:
bankloan_data = bankloan_data.loc[(bankloan_data['Income'] < 300) & (bankloan_data['Creddebt'] < 15) & (bankloan_data['OthDebt'] < 30)]

We can check now how the data looks like.

In [ ]:
fig, ax = plt.subplots(figsize=(10,5))
a = sns.violinplot(x='variable', y='value', data=pd.melt(bankloan_data.iloc[:, np.r_[1,3:9]]), ax=ax)
a.set_xticklabels(a.get_xticklabels(), rotation=90);

Much better, although income still has a large outlier. This plot might be misleading though, as the magnitud of the data is relevant. We might want to recheck this after normalizing the data.

## Normalization

Most models require the inputs to be in the same scale, this is called **normalization**. It is very important for most models... except for credit scoring as we will use Weight of Evidence (see the lecture!). Still, this is fairly simple to do in Pandas, being smart about what columns we select and **mapping** a function to those columns.

The most traditional mapping is the zscore. We can use scipy's version of it. 



In [ ]:
from scipy.stats import zscore

We now select all numeric columns, except the customer one. We can do this with ```select_dtypes```.

In [ ]:
# Identify the numerical columns
numeric_cols = bankloan_data.select_dtypes(include=[np.number]).columns
numeric_cols 

In [ ]:
# Remove the first and last one
numeric_cols = numeric_cols[1:-1]

In [ ]:
# Apply the zscore function to all data
bankloan_data[numeric_cols] = bankloan_data[numeric_cols].apply(zscore)

Now the data should look much better.

In [ ]:
fig, ax = plt.subplots(figsize=(10,5))
a = sns.violinplot(x='variable', y='value', data=pd.melt(bankloan_data.iloc[:, np.r_[1,3:9]]), ax=ax)
a.set_xticklabels(a.get_xticklabels(), rotation=90);

The dataset looks really clean now. We are ready to use further models! To be 100% purist **you should first split between train and test set** before doing all of these analyses (so you should use the median of the train set). Please remember that when solving your coursework.

## Self-study: Dummy coding

The final step will be transform the categorical variables to dummy variables. Again, this is **not necessary for credit scoring** as the WoE transform is much more robust. The best strategy here is to follow the 5% rule:

1. Aggregate by expert judgement (i.e. your opinion) until each group has more than 5% of data (1% if very large data) and there are all classes (by target variable) present.

2. Use target variable percentage (or classification trees) until you get a proper number of groups.

To check the number of cases for each default level, we can calculate a crosstab.

In [ ]:
pd.crosstab(bankloan_data['Education'], bankloan_data['Default'])

Now we can calculate the mean of each educational level.

In [ ]:
bankloan_data.groupby('Education', as_index=False).agg({'Default': ["mean"]})

We could, if we wanted to, group together some educational levels, such as '1upInc', 'Med', and 'Posg'. If not, we can create dummy variables directly with Pandas ```get_dummies```

In [ ]:
bankloan_data = pd.get_dummies(bankloan_data)
bankloan_data.describe()

Now we are ready to apply general models!